In [1]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [2]:
# all imports
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import json
import time

import pandas as pd

from multiprocessing import Pool

import os

In [3]:
airbnb_url = 'https://www.airbnb.es/s/Barcelona--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-02-01&monthly_length=3&price_filter_input_type=0&channel=EXPLORE&source=structured_search_input_header&search_type=autocomplete_click&query=Barcelona%2C%20Espa%C3%B1a&price_filter_num_nights=5&date_picker_type=calendar&place_id=ChIJZb1_yQvmpBIRsMmjIeD6AAM'

In [4]:
soup = BeautifulSoup(requests.get(airbnb_url).content, 'html.parser')

In [5]:
listings = soup.find_all('div', 'cy5jw6o')

In [6]:
listings[1].get_text()

"Apartamento en L'Hospitalet de LlobregatApartamento EuropaFira AirportAnfitrión particularAnfitrión particular1–6 abr1–6 abr67\xa0€\xa0noche67\xa0€ por noche · 331\xa0€ en total331\xa0€ en totalConsulta el desglose del precioValoración media de 2,67 sobre 5, 3\xa0evaluaciones2,67 (3)"

In [7]:
# First Generation :)
def extract_basic_features(listing_html):
    features_dict = {}


    name = listing_html.find("div", {"class": "t1jojoys"}).get_text()
    header = listing_html.find("span", {"class": "t6mzqp7"}).get_text()
    url = listing_html.find('a').get('href')


    features_dict['name'] = name
    features_dict['header'] = header
    features_dict['url'] = url

    return features_dict

In [8]:
extract_basic_features(listings[0])

{'name': 'Habitación en Barcelona',
 'header': 'Habitación privada individual',
 'url': '/rooms/873241702426073436?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1632008954&check_in=2024-04-01&check_out=2024-04-06&source_impression_id=p3_1711642435_LcxWqfOTC76QDwIp&previous_page_section_name=1000&federated_search_id=99506573-0968-4734-8091-c13e403bb5ef'}

In [9]:
# too many separate extractions
RULES_SEARCH_PAGE = {
    'url': {'tag': 'a', 'get': 'href'},
    'name': {'tag': 'div', 'class': 't1jojoys'},
    'header': {'tag': 'span', 'class': 't6mzqp7'},
    'rating_n_reviews': {'tag': 'span', 'class': 'ru0q88m'},
    'price': {'tag': 'span', 'class': 'a8jt5op'},
    'superhost': {'tag': 'span', 'class': 'dir dir-ltr'},
}

In [10]:
# Third Generation :)
def extract_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]

    # 3. Get text
    if 'get' in params:
        output = element.get(params['get'])
    else:
        output = element.get_text()

    return output

In [11]:
print(extract_element(listings[13], RULES_SEARCH_PAGE['header']))
print(extract_element(listings[13], RULES_SEARCH_PAGE['url']))
print(extract_element(listings[13], RULES_SEARCH_PAGE['name']))
print(extract_element(listings[13], RULES_SEARCH_PAGE['rating_n_reviews']))
print(extract_element(listings[13], RULES_SEARCH_PAGE['price']))
print(extract_element(listings[13], RULES_SEARCH_PAGE['superhost']))


Habitación individual, con baño y salón privados!!
/rooms/15813808?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=315921134&check_in=2024-09-14&check_out=2024-09-19&source_impression_id=p3_1711642435_2NFuv6HyOgF9JmC5&previous_page_section_name=1000&federated_search_id=99506573-0968-4734-8091-c13e403bb5ef
Habitación en Barcelona
4,93 (108)
Recomendación del viajero
Quédate con Elena


In [12]:
# for feature in RULES_SEARCH_PAGE:
 #   print(f"{feature}: {extract_element(listings[13], RULES_SEARCH_PAGE[feature])}")

In [13]:
for feature in RULES_SEARCH_PAGE:
    try:
        print(f"{feature}: {extract_element(listings[0], RULES_SEARCH_PAGE[feature])}")
    except:
        print(f"{feature}: empty")

url: /rooms/873241702426073436?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1632008954&check_in=2024-04-01&check_out=2024-04-06&source_impression_id=p3_1711642435_LcxWqfOTC76QDwIp&previous_page_section_name=1000&federated_search_id=99506573-0968-4734-8091-c13e403bb5ef
name: Habitación en Barcelona
header: Habitación privada individual
rating_n_reviews: 4,54 (41)
price: Quédate con Santos
superhost: Quédate con Santos


In [14]:
airbnb_url

'https://www.airbnb.es/s/Barcelona--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-02-01&monthly_length=3&price_filter_input_type=0&channel=EXPLORE&source=structured_search_input_header&search_type=autocomplete_click&query=Barcelona%2C%20Espa%C3%B1a&price_filter_num_nights=5&date_picker_type=calendar&place_id=ChIJZb1_yQvmpBIRsMmjIeD6AAM'

In [15]:
# let's finally write this function
def get_listings(search_page):
    soup = BeautifulSoup(requests.get(search_page).content, 'html.parser')
    listings = soup.find_all('div', 'cy5jw6o')

    return listings

In [16]:
# it works
len(get_listings(airbnb_url))

18

In [17]:
# let's try next page
new_url = airbnb_url + '&items_offset=20'
len(get_listings(new_url))

18

In [18]:
# checking the content, if the data is there
print(extract_element(get_listings(airbnb_url)[0], RULES_SEARCH_PAGE['name']))
print(extract_element(get_listings(new_url)[0], RULES_SEARCH_PAGE['name']))

Loft en Eixample
Apto. residencial en Barcelona


In [19]:
# let's iterate through all 15 pages
all_listings = []
for i in range(15):
    offset = 20 * i
    new_url = airbnb_url + f'&items_offset={offset}'
    new_listings = get_listings(new_url)
    all_listings.extend(new_listings)

    # let's check if it's scraping
    print(len(all_listings))

18
36
54
72
90
108
126
144
162
180
198
216
234
252
270


In [20]:
# why? maybe Airbnb tries to prevent scraping
# let's wait a couple of seconds after every iteration
import time

all_listings = []
for i in range(15):
    offset = 20 * i
    new_url = airbnb_url + f'&items_offset={offset}&section_offset=3'
    new_listings = get_listings(new_url)
    all_listings.extend(new_listings)

    # let's check if it's scraping
    print(len(all_listings))

    time.sleep(2)

18
36
54
72
90
108
126
144
162
180
198
216
234
252
270


In [21]:
# another random check, if the data is there
print(extract_element(all_listings[113], RULES_SEARCH_PAGE['name']))

Habitación en Barcelona


In [22]:
# 1. build all urls
def build_urls(main_url, listings_per_page=20, pages_per_location=15):
    url_list = []
    for i in range(pages_per_location):
        offset = listings_per_page * i
        url_pagination = main_url + f'&items_offset={offset}'
        url_list.append(url_pagination)

    return url_list

In [23]:
# build a list of URLs
url_list = build_urls(airbnb_url)

In [24]:
url_list

['https://www.airbnb.es/s/Barcelona--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-02-01&monthly_length=3&price_filter_input_type=0&channel=EXPLORE&source=structured_search_input_header&search_type=autocomplete_click&query=Barcelona%2C%20Espa%C3%B1a&price_filter_num_nights=5&date_picker_type=calendar&place_id=ChIJZb1_yQvmpBIRsMmjIeD6AAM&items_offset=0',
 'https://www.airbnb.es/s/Barcelona--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-02-01&monthly_length=3&price_filter_input_type=0&channel=EXPLORE&source=structured_search_input_header&search_type=autocomplete_click&query=Barcelona%2C%20Espa%C3%B1a&price_filter_num_nights=5&date_picker_type=calendar&place_id=ChIJZb1_yQvmpBIRsMmjIeD6AAM&items_offset=20',
 'https://www.airbnb.es/s/Barcelona--Espa%C3%B1a/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%

In [25]:
# safe function to extract all features from one page
def extract_page_features(soup, rules):
    features_dict = {}
    for feature in rules:
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'

    return features_dict

In [26]:
extract_page_features(listings[0], RULES_SEARCH_PAGE)

{'url': '/rooms/873241702426073436?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1632008954&check_in=2024-04-01&check_out=2024-04-06&source_impression_id=p3_1711642435_LcxWqfOTC76QDwIp&previous_page_section_name=1000&federated_search_id=99506573-0968-4734-8091-c13e403bb5ef',
 'name': 'Habitación en Barcelona',
 'header': 'Habitación privada individual',
 'rating_n_reviews': '4,54 (41)',
 'price': 'Quédate con Santos',
 'superhost': 'Quédate con Santos'}

In [27]:
# 2. Iteratively scrape pages
def process_search_pages(url_list):
    features_list = []
    for page in url_list:
        listings = get_listings(page)
        for listing in listings:
            features = extract_page_features(listing, RULES_SEARCH_PAGE)
            features_list.append(features)

    return features_list

In [28]:
# try for one page
base_features = process_search_pages(url_list)

In [29]:
base_features

[{'url': '/rooms/873241702426073436?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1632008954&check_in=2024-04-01&check_out=2024-04-06&source_impression_id=p3_1711642514_Ti%2FX%2B4XVAim3M38F&previous_page_section_name=1000&federated_search_id=ba5ce62f-4f44-4df3-bb7d-695b0a1b75a7',
  'name': 'Habitación en Barcelona',
  'header': 'Habitación privada individual',
  'rating_n_reviews': '4,54 (41)',
  'price': 'Quédate con Santos',
  'superhost': 'Quédate con Santos'},
 {'url': '/rooms/829538139176678540?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-05-26&check_out=2024-05-31&source_impression_id=p3_1711642514_i0U3ohRLGVGXdY%2Bm&previous_page_section_name=1000&federated_search_id=ba5ce62f-4f44-4df3-bb7d-695b0a1b75a7',
  'name': 'Alojamiento en Barcelona',
  'header': 'Habitación con comedor propio',
  'rating_n_reviews': 'empty',
  'price': 'Anfitrión particular',
  'superhost': 'Anfitrión particular'},
 {'

In [30]:
df = pd.DataFrame(base_features)
df

,url,name,header,rating_n_reviews,price,superhost
0,/rooms/873241702426073436?adults=1&category_ta...,Habitación en Barcelona,Habitación privada individual,"4,54 (41)",Quédate con Santos,Quédate con Santos
1,/rooms/829538139176678540?adults=1&children=0&...,Alojamiento en Barcelona,Habitación con comedor propio,empty,Anfitrión particular,Anfitrión particular
2,/rooms/1110645966155434883?adults=1&children=0...,Alojamiento en Barcelona,Hermoso estudio entre Navas y Camp del Arpa,Nuevo,Anfitrión particular,Anfitrión particular
3,/rooms/1026839665506453132?adults=1&category_t...,Habitación en Barcelona,Habitación tranquila y céntrica,"4,89 (18)",Recomendación del viajero,Quédate con Glòria
4,/rooms/1112329611690731083?adults=1&children=0...,Alojamiento en Barcelona,Habitación privada sin ascensor baño compartido,Nuevo,Anfitrión particular,Anfitrión particular
...,...,...,...,...,...,...
265,/rooms/5037632?adults=1&children=0&enable_m3_p...,Apartamento en Gràcia,Piso con vistas A los naranjos. LUGAR para estar,"4,84 (213)",Superanfitrión,Gestionado por una empresa
266,/rooms/559007133975079392?adults=1&children=0&...,Apartamento en Eixample,New flat between plaza españa and plaza catalonia,"4,88 (69)",Recomendación del viajero,Anfitrión: Sandra Teófila
267,/rooms/19043388?adults=1&children=0&enable_m3_...,Apartamento en Eixample,Apartamento de dos dormitorios en el paseo de ...,"4,87 (54)",Gestionado por una empresa,Gestionado por una empresa
268,/rooms/8181239?adults=1&children=0&enable_m3_p...,Apto. residencial en Eixample,BRUC 69 - PASEO DE GRACIA,"4,82 (396)",Anfitrión: Maria,Anfitrión: Maria


In [31]:
df['ciudad'] = df['name'].str.replace('Villa en |Cueva en |Vivienda en |Apartamento en|Habitación en ', '', regex=True)
df

,url,name,header,rating_n_reviews,price,superhost,ciudad
0,/rooms/873241702426073436?adults=1&category_ta...,Habitación en Barcelona,Habitación privada individual,"4,54 (41)",Quédate con Santos,Quédate con Santos,Barcelona
1,/rooms/829538139176678540?adults=1&children=0&...,Alojamiento en Barcelona,Habitación con comedor propio,empty,Anfitrión particular,Anfitrión particular,Alojamiento en Barcelona
2,/rooms/1110645966155434883?adults=1&children=0...,Alojamiento en Barcelona,Hermoso estudio entre Navas y Camp del Arpa,Nuevo,Anfitrión particular,Anfitrión particular,Alojamiento en Barcelona
3,/rooms/1026839665506453132?adults=1&category_t...,Habitación en Barcelona,Habitación tranquila y céntrica,"4,89 (18)",Recomendación del viajero,Quédate con Glòria,Barcelona
4,/rooms/1112329611690731083?adults=1&children=0...,Alojamiento en Barcelona,Habitación privada sin ascensor baño compartido,Nuevo,Anfitrión particular,Anfitrión particular,Alojamiento en Barcelona
...,...,...,...,...,...,...,...
265,/rooms/5037632?adults=1&children=0&enable_m3_p...,Apartamento en Gràcia,Piso con vistas A los naranjos. LUGAR para estar,"4,84 (213)",Superanfitrión,Gestionado por una empresa,Gràcia
266,/rooms/559007133975079392?adults=1&children=0&...,Apartamento en Eixample,New flat between plaza españa and plaza catalonia,"4,88 (69)",Recomendación del viajero,Anfitrión: Sandra Teófila,Eixample
267,/rooms/19043388?adults=1&children=0&enable_m3_...,Apartamento en Eixample,Apartamento de dos dormitorios en el paseo de ...,"4,87 (54)",Gestionado por una empresa,Gestionado por una empresa,Eixample
268,/rooms/8181239?adults=1&children=0&enable_m3_p...,Apto. residencial en Eixample,BRUC 69 - PASEO DE GRACIA,"4,82 (396)",Anfitrión: Maria,Anfitrión: Maria,Apto. residencial en Eixample


In [32]:
ruta = 'Alojamientos_Barcelona_Airbnb.csv'
df.to_csv(ruta,index=False)